# TD analyse de graphe botnet

In [ ]:
from collections import defaultdict
import numpy as np
import random

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import plotly.express as px

# I. Statistiques descriptives du jeu de données ISOT

In [ ]:
EDGE_PATH = '../../data/isot_edges_unique_with_dest_port.csv'
NODE_PATH = '../../data/isot_nodes.csv'

#### 1) Charger le fichier de liens avec Pandas

In [ ]:
df_edges = pd.read_csv(EDGE_PATH)
df_edges

#### 2) Afficher l'histogramme de la colonne 'packets' (le nombre de connexions de la source à la target). Que constatez-vous ?

In [ ]:
df_edges.describe()

In [ ]:
df_edges['packets'].hist(bins=100, log=True)

Certains liens sont très fréquents, ce qui veut dire que certaines IP ont effectuées un très grand nombre de connexions à d'autres.

#### 3) Afficher l'histogramme de la colonne 'length' (le poids total des échanges réalisés lors des connexons). Que constatez-vous ?

In [ ]:
df_edges['length'].hist(bins=100, log=True)

Quelques packets transportent de grande quantitées de données.

#### 4) Afficher un scatter plot des liens avec en abcisse le nombre de connexions ('packets') et en ordonné le poids total des échanges ('length'). Que remarquez-vous ?

In [ ]:
fig = plt.figure(figsize=(20, 8))
plt.scatter(df_edges['packets'], df_edges['length'])
plt.xlabel("Nombre de connexions")
plt.ylabel("Poids total des échanges")
plt.legend(loc='upper left')
plt.title('Liens en fonction du nombre de connexions et du poids de leurs échanges')
plt.show()

In [ ]:
px.scatter(
    df_edges, 'packets', 'length',
    title='Liens en fonction du nombre de connexions et du poids de leurs échanges',
    color='malicious',
    #trendline="ols"
)

- On constate un très fort nombre de connexions entre quelques IPs en particulier.
- On peut constater que de gros volumes de données sont échangés lors de connexions de quelques IPs en particulier.
- On constate également une correlation positive entre le nombre de connexions et la taille des volumes de données échangées via une relation linéaire.

In [ ]:
df_edges.port_destination.hist(bins=100)

#### 5) En prenant en compte le graphique précédent, afficher les liens qui vous semblent intéressants

- On constate un très fort nombre de connexions entre quelques IPs en particulier :

Toutes ces connections proviennent de bots...

- On peut constater que de gros volumes de données sont échangés lors de connexions quelques IPs en particulier :

In [ ]:
df_edges[
    (df_edges['length'] >= 0.5*1e7)
]

Toutes ces connections proviennent de bots...

# II. Construction du graphe: IP Source -> port IP Dest <- IP Dest

**Création de la colonne 'ip_port_destination'**

Un noeud pour chaque port ouvert par ip source unique par ip destination.

In [ ]:
df_edges['port_destination'] = df_edges['port_destination'].astype(str)
df_edges['ip_port_destination'] = df_edges['port_destination'] + ' (' + df_edges['ip_source'] + '-' + df_edges['ip_destination'] + ')'
df_edges

**Creation d'une liste de liens entre les Target et leurs ports interrogés**

In [ ]:
# on crée une copie du dataframe `df_edges`
df_edges_target_ip = df_edges.copy()

# suppression des doublons
df_edges_target_ip = df_edges_target_ip[['ip_destination', 'ip_port_destination']].drop_duplicates()

# ajout d'une colonne `edge_type`
df_edges_target_ip['edge_type'] = 'HAS_PORT'

# renommage des colonnes
df_edges_target_ip.rename({'ip_destination': 'Source', 'ip_port_destination': 'Target'}, inplace=True, axis=1)

df_edges_target_ip

In [ ]:
# nb de ports ouverts par IP
df_edges_target_ip['Source'].value_counts()

**Construction des liens de entre les sources et les ports des targets**

In [ ]:
# on crée une copie du dataframe `df_edges`
df_edges_source_target_ip = df_edges.copy()

# ajout d'une colonne `edge_type`
df_edges_source_target_ip['edge_type'] = 'HAS_CONNECTION'

# suppression de colonnes
df_edges_source_target_ip.drop(['ip_destination', 'port_destination'], axis=1, inplace=True)

# renommage des colonnes
df_edges_source_target_ip.rename({'ip_source': 'Source', 'ip_port_destination': 'Target'}, inplace=True, axis=1)

df_edges_source_target_ip

**Détection des noeuds qui envoies le plus de packets à des ports différents (out degree)**

Indice : sur `df_edges_source_target_ip`, utiliser la fonction `value_counts` de Pandas sur la colonne `Source` pour voir les noeuds qui ont le plus de liens sortants. Que constatez-vous ?

In [ ]:
df_edges_source_target_ip['Source'].value_counts()

DNS server des auteurs du dataset : 192.168.50.88

**Fusion df_edges_source_target_ip et df_edges_target_ip**

In [ ]:
df_edges_merged = pd.concat([df_edges_source_target_ip, df_edges_target_ip])
print(df_edges_merged['edge_type'].value_counts())
df_edges_merged

**Parmi tous les liens de `df_edges_merged`, afficher le nombre de liens `malicous` et le nombre de liens qui ne le sont pas**

In [ ]:
df_edges_merged['malicious'].value_counts()

**Construction du graph**

In [ ]:
# build a graph from edges
G = nx.from_pandas_edgelist(
    df_edges_merged,
    source='Source',
    target='Target',
    edge_attr=['packets', 'edge_type', 'malicious', 'length', 'mean_length_by_packet'],
    create_using=nx.DiGraph()
)
# add nodes attributes
df_nodes = pd.read_csv(NODE_PATH)
nx.set_node_attributes(G, df_nodes.set_index('ip').to_dict('index'))

G.number_of_nodes(), G.number_of_edges()

**Afficher les composantes connexes du graphe**

Indice : si vous avez installer `pygraphviz`, utiliser `nx.nx_pydot.graphviz_layout(G, prog="dot")`, sinon `nx.fruchterman_reingold_layout` et `nx.draw`

In [ ]:
G = G.to_undirected()
components_nodes = sorted(nx.connected_components(G), key=len, reverse=True)

for i in range(len(components_nodes)):
    # get the n component
    graph_component = G.subgraph(components_nodes[i])
    print(f"{i+1} first component ({len(graph_component)} nodes)")

    # subsample if needed
    k = 2000
    if len(graph_component) > k:
        sampled_edges = random.sample(list(graph_component.edges), k)
        graph_component = graph_component.edge_subgraph(sampled_edges)

    # displaying the 
    fig = plt.figure(figsize=(20, 20))
    pos = nx.spring_layout(graph_component)
    plt.axis("off")
    nx.draw_networkx_nodes(graph_component, pos, node_size=20)
    nx.draw_networkx_edges(graph_component, pos, alpha=0.4)
    plt.show()

**Afficher un egographe de niveau 2**

In [ ]:
ego = nx.ego_graph(G, '137 (192.168.50.11-192.168.50.255)', radius=2)
print(len(ego.nodes()))
fig = plt.figure(figsize=(20, 20))
pos = nx.spring_layout(ego)
plt.axis("off")
nx.draw_networkx_nodes(ego, pos, node_size=20)
nx.draw_networkx_edges(ego, pos, alpha=0.4)
plt.show()

**Sauvegarde du graphe**

Indice : utiliser `nx.write_gml`

In [ ]:
nx.write_gml(G, '../../data/isot_gephi.gml')

**Mise en forme sur Gephi**